# Import data

In [ ]:
import pandas as pd
import numpy as np

nfh_input_file_path='nfh_raw_measurements.parquet'
enelogic_input_path='rhc_enelogic_monthly_export.parquet'
remeha_input_file_path='remeha_export.parquet'
rhc_output_file_path='rhc_raw_measurements.parquet'
home_data_file_path = "home_data.parquet"

# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

import sys
sys.path.append('../data/')
sys.path.append('../view/')

%load_ext autoreload


%matplotlib inline
%matplotlib widget
import pylab as plt
import itertools
from plotter import Plot
from tqdm.notebook import tqdm
import historicdutchweather
from measurements import Measurements, WeatherMeasurements

from urllib.error import HTTPError  # Import HTTPError from urllib.error

## Read NeedForHeat data

In [ ]:
%%time
# Attempt to read the Parquet file
try:
    df_meas_nfh = pd.read_parquet(
        nfh_input_file_path, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")


In [ ]:
df_meas_nfh.info()

In [ ]:
df_meas_nfh.index.dtypes

### Add unit to hearbeat property

In [ ]:
list(df_meas_nfh.index.get_level_values('property').unique())

In [ ]:
%%time

# Reset index to modify column names
df_meas_nfh.reset_index(inplace=True)

# Replace 'heartbeat' values with 'heartbeat__0' using map
df_meas_nfh['property'] = df_meas_nfh['property'].map(lambda x: 'heartbeat__0' if x == 'heartbeat' else x)

# # Convert relevant columns to categorical type
df_meas_nfh['source_category'] = df_meas_nfh['source_category'].astype('category')
df_meas_nfh['source_type'] = df_meas_nfh['source_type'].astype('category')
df_meas_nfh['property'] = df_meas_nfh['property'].astype('category')

# Set the index again
df_meas_nfh.set_index(['id', 'source_category', 'source_type', 'timestamp', 'property'], inplace=True)

In [ ]:
list(df_meas_nfh.index.get_level_values('property').unique())

In [ ]:
df_meas_nfh.index.dtypes

In [ ]:
df_meas_nfh

## Read Enelogic data

In [ ]:
%%time
# Attempt to read the Parquet file
try:
    df_meas_enelogic = pd.read_parquet(
        enelogic_input_path, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")


In [ ]:
df_meas_enelogic.info()

In [ ]:
df_meas_enelogic.index.dtypes

In [ ]:
df_meas_enelogic

In [ ]:
%%time
df_meas_enelogic['value'] = df_meas_enelogic['value'].astype(str)

In [ ]:
df_meas_enelogic.info()

## Read Remeha data

In [ ]:
%%time
# Attempt to read the Parquet file
try:
    df_meas_remeha = pd.read_parquet(
        remeha_input_file_path, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")


In [ ]:
df_meas_remeha.info()

In [ ]:
df_meas_remeha.index.dtypes

In [ ]:
df_meas_remeha

In [ ]:
%%time
df_meas_remeha['value'] = df_meas_remeha['value'].astype(str)

In [ ]:
df_meas_remeha.info()

## Merge measurements


In [ ]:
%%time
df_meas = pd.concat([df_meas_nfh, df_meas_enelogic, df_meas_remeha])

In [ ]:
df_meas.index.dtypes

In [ ]:
%%time
# Reset the index to convert the MultiIndex back to columns
df_meas_reset = df_meas.reset_index()

# Convert relevant columns to categorical type
df_meas_reset['source_category'] = df_meas_reset['source_category'].astype('category')
df_meas_reset['source_type'] = df_meas_reset['source_type'].astype('category')
df_meas_reset['property'] = df_meas_reset['property'].astype('category')

# Set the columns back to a MultiIndex
df_meas = df_meas_reset.set_index(['id', 'source_category', 'source_type', 'timestamp', 'property'])


In [ ]:
df_meas.index.dtypes

## Get and merge geospatially interpolated KNMI weather measurements

In [ ]:
%%time
# read home data from Parquet file
try:
    df_homes = pd.read_parquet(
        home_data_file_path, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")


In [ ]:
%%time
%autoreload 2
df_meas_weather = WeatherMeasurements.get_nfh_weather_measurements(df_meas, df_homes)

In [ ]:
df_meas_weather.info()

In [ ]:
df_meas_weather.index.dtypes

In [ ]:
# Describe statistics across all properties
df_meas_weather['value'].unstack('property').describe().T

## Merge weather measurements


In [ ]:
%%time
df_meas_weather['value'] = df_meas_weather['value'].astype(str)

In [ ]:
%%time
df_meas = pd.concat([df_meas, df_meas_weather])

In [ ]:
df_meas.index.dtypes

In [ ]:
%%time
# Reset the index to convert the MultiIndex back to columns
df_meas_reset = df_meas.reset_index()

# Convert relevant columns to categorical type
df_meas_reset['source_category'] = df_meas_reset['source_category'].astype('category')
df_meas_reset['source_type'] = df_meas_reset['source_type'].astype('category')
df_meas_reset['property'] = df_meas_reset['property'].astype('category')

# Set the columns back to a MultiIndex
df_meas = df_meas_reset.set_index(['id', 'source_category', 'source_type', 'timestamp', 'property'])

In [ ]:
df_meas.index.dtypes

## Initial exploration: size, ids, start & stop times per id

In [ ]:
df_meas.info()

In [ ]:
df_meas.index.dtypes

In [ ]:
print(f"len(df_meas): {len(df_meas):_}")

In [ ]:
%%time
# deduplicate the measurements
df_meas = df_meas.reset_index().drop_duplicates().set_index(['id', 'source_category', 'source_type', 'timestamp', 'property']).sort_index()

In [ ]:
print(f"len(df_meas): {len(df_meas):_}")

In [ ]:
list(df_meas.index.get_level_values('id').unique())

In [ ]:
len(df_meas.index.get_level_values('id').unique())

In [ ]:
%%time
df_meas.reset_index().groupby(['id', 'source_type'], observed=True)['timestamp'].agg(['min', 'max'])

In [ ]:
print(f"df_meas['value'].count(): {df_meas['value'].count():_}")

In [ ]:
df_meas.duplicated().any()

In [ ]:
df_meas.info()

In [ ]:
df_meas.describe()

In [ ]:
df_meas

In [ ]:
list(df_meas.index.get_level_values('source_category').unique())

In [ ]:
list(df_meas.index.get_level_values('source_type').unique())

In [ ]:
list(df_meas.index.get_level_values('property').unique())

In [ ]:
df_meas.groupby(['source_type'], observed=True).size()

In [ ]:
df_meas.groupby(['source_category', 'source_type', 'property'], observed=True).size()

In [ ]:
print(f"df_meas.count().sum(): {df_meas.count().sum():_}")

## Write to parquet file(s)

In [ ]:
df_meas.index.dtypes

In [ ]:
%%time 
# Convert the 'value' column to string type
df_meas['value'] = df_meas['value'].astype(str)

In [ ]:
%%time 
df_meas.to_parquet(rhc_output_file_path, index=True, engine='pyarrow')

In [ ]:
# # uncomment this code to enable it 
# %%time 
# for home_id in tqdm(df_meas.index.get_level_values('id').unique()):
#     df_meas.xs(home_id, drop_level=False).to_parquet(f'{home_id}_raw_measurements.parquet', index=True, engine='pyarrow')